# MNIST dataset - Neural Netowrk 
Built the neural network from scratch... Got an accuracy upto 98.5% (without minibatches)

*> **Note:** `cupy` is simalar to 'numpy' with GPU support*

**Net Schema:** INP(784) -> [RELU(128) + tanh(64)](192) -> RELU(48)) -> softmax(10)  with regularisation, inverted dropout, etc.

*This is not a formal documentation!*

**Import libraries**

In [0]:
import cupy as np
import matplotlib.pyplot as plt

**Import Dataset**

*Change the path!*

In [0]:
import gzip, pickle
with gzip.open('/content/drive/My Drive/colab/mnist.pkl.gz','rb') as ff :
  u = pickle._Unpickler( ff )
  u.encoding = 'latin1'
  train, val, test = u.load()

**Minor changes to dataset**

In [0]:
X_train = train[0].T
y_train = train[1].reshape(1,train[1].shape[0])

X_val = val[0].T
y_val = val[1].reshape(1,val[1].shape[0])

X_test = test[0].T
y_test = test[1].reshape(1,test[1].shape[0])

Y_train = np.zeros((10,y_train.shape[1]))
Y_val = np.zeros((10,y_val.shape[1]))
Y_test = np.zeros((10,y_test.shape[1]))

Y_train[y_train,np.arange(y_train.shape[1])]=1
Y_val[y_val,np.arange(y_val.shape[1])]=1
Y_test[y_test,np.arange(y_test.shape[1])]=1

In [0]:
X_train = np.array(X_train)
y_train = np.array(y_train)
Y_train = np.array(Y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)
Y_val = np.array(Y_val)

X_test = np.array(X_test)
y_test = np.array(y_test)
Y_test = np.array(Y_test)

**Useful methods**

In [0]:
def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def tanh_(X):
  return 1-tanh(X)**2

def relu(X):
  return np.maximum(0,X)

def relu_(X):
  return (X>0).astype(int)


def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X),axis=0,keepdims=True)
    return expo/expo_sum

**Initialise parameters**

Used Xavier & He's method for weights and zeros for biases

In [0]:
def init_param():
  np.random.seed(4)
  
  W11 = np.random.randn(128,784) * np.sqrt(2/784)
  b11 = np.zeros((128,1))
  # b11 = np.random.randn(128,1) * np.sqrt(1/784)
  
  W12 = np.random.randn(64,784) * np.sqrt(2/784)
  b12 = np.zeros((64,1))
  # b12 = np.random.randn(64,1) * np.sqrt(1/784)
  
  W2 = np.random.randn(48,192) * np.sqrt(1/192)
  b2 = np.zeros((48,1))
  # b2 = np.random.randn(48,1) * np.sqrt(1/192)
  
  W3 = np.random.randn(10,48) * np.sqrt(2/48)
  b3 = np.zeros((10,1))
  # b3 = np.random.randn(10,1) * np.sqrt(1/48)

  np.cuda.Stream.null.synchronize()

  return W11,b11,W12,b12,W2,b2,W3,b3

**Forward prop**

[RELU tanh] -> RELU -> softmax

  [128 64]  -> [48] ->   [10]

In [0]:
def fwd_prop(X,Y,W11,b11,W12,b12,W2,b2,W3,b3,keep_prob_11=1,keep_prob_12=1,keep_prob_2=1,reg_fact_3=1.2,reg_fact_2=1.2,reg_fact_12=0.9,reg_fact_11=1.2,epsilon=10e-8):
  m = X.shape[1]
  A11_,A12_,A2_=0,0,0

  Z11 = np.dot(W11,X) + b11
  Z12 = np.dot(W12,X) + b12
  np.cuda.Stream.null.synchronize()
  
  A11 = relu(Z11)
  A12 = tanh(Z12)
  np.cuda.Stream.null.synchronize()

  if keep_prob_11 < 1:
    A11_ = np.random.rand(A11.shape[0],1)
    np.cuda.Stream.null.synchronize()
    A11_ = A11_<keep_prob_11
    np.cuda.Stream.null.synchronize()
    A11 = np.multiply(A11_,A11)
    np.cuda.Stream.null.synchronize()
    A11 /= keep_prob_11
    np.cuda.Stream.null.synchronize()

  if keep_prob_12 < 1:
    A12_ = np.random.rand(A12.shape[0],1)
    np.cuda.Stream.null.synchronize()
    A12_ = A12_<keep_prob_12
    np.cuda.Stream.null.synchronize()
    A12 = np.multiply(A12_,A12)
    np.cuda.Stream.null.synchronize()
    A12 /= keep_prob_12

  A1 = np.concatenate((A11,A12))
  np.cuda.Stream.null.synchronize()

  Z2 = np.dot(W2,A1) + b2
  np.cuda.Stream.null.synchronize()
  A2 = relu(Z2)
  np.cuda.Stream.null.synchronize()

  if keep_prob_2 < 1:
    A2_ = np.random.rand(A2.shape[0],1)
    np.cuda.Stream.null.synchronize()
    A2_ = A2_<keep_prob_2
    np.cuda.Stream.null.synchronize()
    A2 = np.multiply(A2_,A2)
    np.cuda.Stream.null.synchronize()
    A2 /= keep_prob_2

  Z3 = np.dot(W3,A2) + b3
  np.cuda.Stream.null.synchronize()
  A3 = softmax(Z3)
  np.cuda.Stream.null.synchronize()

  #reg_fact_11 = reg_fact_12 = reg_fact_2 = reg_fact_3 = 0.5 
  cross_entropy = -np.sum(Y*np.log(A3+epsilon))/m
  reg = (reg_fact_3*np.mean(W3**2)+reg_fact_2*np.mean(W2**2)+reg_fact_12*np.mean(W12**2)+reg_fact_11*np.mean(W11**2))/2
  np.cuda.Stream.null.synchronize()

  J = cross_entropy + reg
  np.cuda.Stream.null.synchronize()

  cache = []
  cache.append(A3)
  cache.append(A2)
  cache.append(A12)
  cache.append(A11)
  cache.append(Z3)
  cache.append(Z2)
  cache.append(Z12)
  cache.append(Z11)

  cache.append(A2_)
  cache.append(A12_)
  cache.append(A11_)

  


  return A3,J,cache

**Back Prop**

In [0]:
def bck_prop(X,Y,W11,W12,W2,W3,cache,keep_prob_11=1,keep_prob_12=1,keep_prob_2=1,reg_fact_3=1.2,reg_fact_2=1.2,reg_fact_12=0.9,reg_fact_11=1.2):
  A3 = cache[0]
  A2 = cache[1]
  A12 = cache[2]
  A11 = cache[3]
  Z3 = cache[4]
  Z2 = cache[5]
  Z12 = cache[6]
  Z11 = cache[7]

  A2_ = cache[8]
  A12_ = cache[9]
  A11_ = cache[10]


  # reg_fact_11 = reg_fact_12 = reg_fact_2 = reg_fact_3 = 0.5 

  A1 = np.concatenate((A11,A12))

  m = X.shape[1]

  dZ3 = A3-Y
  np.cuda.Stream.null.synchronize()
  
  dW3 = np.dot(dZ3,A2.T)/m
  db3 = np.sum(dZ3,axis=1,keepdims=True)/m

  dZ2 = np.dot(W3.T,dZ3)*relu_(Z2)
  
  np.cuda.Stream.null.synchronize()

  if keep_prob_2 < 1:
    dZ2 = np.multiply(dZ2,A2_)/keep_prob_2
  
  np.cuda.Stream.null.synchronize()
  
  dW2 = np.dot(dZ2,A1.T)/m
  db2 = np.sum(dZ2,axis=1,keepdims=True)/m

  dZ12 = np.dot(W2.T,dZ2)[Z11.shape[0]:,:]*tanh_(Z12)
  dZ11 = np.dot(W2.T,dZ2)[0:Z11.shape[0],:]*relu_(Z11)
  
  np.cuda.Stream.null.synchronize()

  if keep_prob_12 < 1:
    dZ12 = np.multiply(dZ12,A12_)/keep_prob_12
    
  if keep_prob_11 < 1:
    dZ11 = np.multiply(dZ11,A11_)/keep_prob_11
  
  np.cuda.Stream.null.synchronize()
  

  dW12 = np.dot(dZ12,X.T)/m
  db12 = np.sum(dZ12,axis=1,keepdims=True)/m

  dW11 = np.dot(dZ11,X.T)/m
  db11 = np.sum(dZ11,axis=1,keepdims=True)/m

  np.cuda.Stream.null.synchronize()


  dW11 += reg_fact_11 * W11 /m
  dW12 += reg_fact_12 * W12 /m
  dW2 += reg_fact_2 * W2 /m
  dW3 += reg_fact_3 * W3 /m

  np.cuda.Stream.null.synchronize()

  return dW11,db11,dW12,db12,dW2,db2,dW3,db3

**A step in Adam-*like* optimser**

Does NOT include the scaling factor

In [0]:
def Adam_step(X,v,s,dX,i,learn_rate,beta1,beta2,epsilon):
  v = beta1*v + (1-beta1)*dX
  s = beta2*s + (1-beta2)*(dX**2)

  np.cuda.Stream.null.synchronize()

  #v_ = v/(1-(beta1)**i)
  #s_ = s/(1-(beta2)**i)

  #X-=learn_rate*v
  X -= (learn_rate * v / (np.sqrt(s)+epsilon))
  np.cuda.Stream.null.synchronize()

  return X,v,s


**Train model**

Also check accuracy at each 100 steps

In [0]:
def train_model(X,Y,learn_rate=0.001,iter=1000,reg_fact_3=1.2,reg_fact_2=1.2,reg_fact_12=1.05,reg_fact_11=1.19,keep_prob_11=0.93,keep_prob_12=0.93,keep_prob_2=0.95,beta1=0.9,beta2=0.999,epsilon=10e-8):
  W11,b11,W12,b12,W2,b2,W3,b3 = init_param()  
  np.cuda.Stream.null.synchronize()  
   
  #W11,b11,W12,b12,W2,b2,W3,b3 = upd_param(W11,b11,W12,b12,W2,b2,W3,b3,dW11,db11,dW12,db12,dW2,db2,dW3,db3,learn_rate=learn_rate)

  J_t = 10000

  J=10000
  v_dW11,v_dW12,v_dW2,v_dW3,v_db11,v_db12,v_db2,v_db3,s_dW11,s_dW12,s_dW2,s_dW3,s_db11,s_db12,s_db2,s_db3 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


  for i in range(iter):

    J_t = J

    _,J,cache = fwd_prop(X_train,Y_train,W11,b11,W12,b12,W2,b2,W3,b3,keep_prob_11=keep_prob_11,keep_prob_12=keep_prob_12,keep_prob_2=keep_prob_2,reg_fact_3=reg_fact_3,reg_fact_2=reg_fact_2,reg_fact_12=reg_fact_12,reg_fact_11=reg_fact_11)
    np.cuda.Stream.null.synchronize()

    dW11,db11,dW12,db12,dW2,db2,dW3,db3 = bck_prop(X_train,Y_train,W11,W12,W2,W3,cache,keep_prob_11=keep_prob_11,keep_prob_12=keep_prob_12,keep_prob_2=keep_prob_2,reg_fact_3=reg_fact_3,reg_fact_2=reg_fact_2,reg_fact_12=reg_fact_12,reg_fact_11=reg_fact_11)
    np.cuda.Stream.null.synchronize()

    '''
    W11 -= learn_rate * dW11
    b11 -= learn_rate * db11
    W12 -= learn_rate * dW12
    b12 -= learn_rate * db12
    W2 -= learn_rate * dW2
    b2 -= learn_rate * db2
    W3 -= learn_rate * dW3
    b3 -= learn_rate * db3
    '''
    #if (i%1000==0):
    #  learn_rate*=np.sqrt(1/((i//1000)+1))

    W11,v_dW11,s_dW11 = Adam_step(W11,v_dW11,s_dW11,dW11,i+1,learn_rate,beta1,beta2,epsilon)
    W12,v_dW12,s_dW12 = Adam_step(W12,v_dW12,s_dW12,dW12,i+1,learn_rate,beta1,beta2,epsilon)
    W2,v_dW2,s_dW2 = Adam_step(W2,v_dW2,s_dW2,dW2,i+1,learn_rate,beta1,beta2,epsilon)
    W3,v_dW3,s_dW3 = Adam_step(W3,v_dW3,s_dW3,dW3,i+1,learn_rate,beta1,beta2,epsilon)

    b11,v_db11,s_db11 = Adam_step(b11,v_db11,s_db11,db11,i+1,learn_rate,beta1,beta2,epsilon)
    b12,v_db12,s_db12 = Adam_step(b12,v_db12,s_db12,db12,i+1,learn_rate,beta1,beta2,epsilon)
    b2,v_db2,s_db2 = Adam_step(b2,v_db2,s_db2,db2,i+1,learn_rate,beta1,beta2,epsilon)
    b3,v_db3,s_db3 = Adam_step(b3,v_db3,s_db3,db3,i+1,learn_rate,beta1,beta2,epsilon)
    

    np.cuda.Stream.null.synchronize()

    #if J>J_t:
      #print("\t\t Warning!! J exceeded prev value")
      #print("\t\tWarning at ",i,':\t J=',J,'exceeded previous value \t [ J_prev =',J_t,']')
      #break
    
    if i%1000==0:
      reg_fact_11*=np.sqrt(i/1000+2)/2
      reg_fact_12*=np.sqrt(i/1000+2)/2
      reg_fact_2*=np.sqrt(i/1000+2)/2
      reg_fact_3*=np.sqrt(i/1000+2)/2
    '''


    if i%1000==0:
      learn_rate*=np.exp(-(i/1000))
    '''
    if i%50==0:
      print("\tIteration ",i,':\t J=',J,end='\t\t')
      ans,_,__ = fwd_prop(X_train,Y_train,W11,b11,W12,b12,W2,b2,W3,b3)
      ans1,_,__ = fwd_prop(X_val,Y_val,W11,b11,W12,b12,W2,b2,W3,b3)
      ans2,_,__ = fwd_prop(X_test,Y_test,W11,b11,W12,b12,W2,b2,W3,b3)
      np.cuda.Stream.null.synchronize()

      ans = np.argmax(ans,axis=0)
      ans1 = np.argmax(ans1,axis=0)
      ans2 = np.argmax(ans2,axis=0)
      np.cuda.Stream.null.synchronize()

      acc = np.mean((ans==y_train).astype(int))
      acc1 = np.mean((ans1==y_val).astype(int))
      acc2 = np.mean((ans2==y_test).astype(int))
      np.cuda.Stream.null.synchronize()

      ACC = [acc,acc1,acc2]

      print('Accuracy: ',ACC)



  return W11,b11,W12,b12,W2,b2,W3,b3   

**Start training in loop**

Trained only once here...

The regularisation parameters & dropout probability given in the respective dictionaries

*Import time from bottom if required!*

In [40]:
st = time.time()

for i in range(1):
  learn_rate = np.random.rand() * -4
  learn_rate = 10 ** learn_rate

  learn_rate = 0.001

  

  reg_param = {'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.07,
    'reg_fact_3': 2.22}
  keep_prob = {'keep_prob_11': 0.8, 'keep_prob_12': 0.85, 'keep_prob_2': 0.9}


  print(">>>>>>>>>>>>>>> reg =",reg_param,"|| keep_prob =",keep_prob)
  W11,b11,W12,b12,W2,b2,W3,b3=train_model(X_train,Y_train,iter=10000,keep_prob_11=keep_prob['keep_prob_11'],keep_prob_12=keep_prob['keep_prob_11'],keep_prob_2=keep_prob['keep_prob_11'],reg_fact_3=reg_param['reg_fact_3'],reg_fact_2=reg_param['reg_fact_2'],reg_fact_12=reg_param['reg_fact_12'],reg_fact_11=reg_param['reg_fact_11'])
  np.cuda.Stream.null.synchronize()
  print('===============',end=' ')

et = time.time()

el = et - st

ans,_,__ = fwd_prop(X_train,Y_train,W11,b11,W12,b12,W2,b2,W3,b3)
ans1,_,__ = fwd_prop(X_val,Y_val,W11,b11,W12,b12,W2,b2,W3,b3)
ans2,_,__ = fwd_prop(X_test,Y_test,W11,b11,W12,b12,W2,b2,W3,b3)
np.cuda.Stream.null.synchronize()

ans = np.argmax(ans,axis=0)
ans1 = np.argmax(ans1,axis=0)
ans2 = np.argmax(ans2,axis=0)
np.cuda.Stream.null.synchronize()

acc = np.mean((ans==y_train).astype(int))
acc1 = np.mean((ans1==y_val).astype(int))
acc2 = np.mean((ans2==y_test).astype(int))
np.cuda.Stream.null.synchronize()

ACC = [acc,acc1,acc2]

print('Accuracy: ',ACC)
print("Time taken: ",el//60,'m ',el%60,'s')






>>>>>>>>>>>>>>> reg = {'reg_fact_11': 1.19, 'reg_fact_12': 1.05, 'reg_fact_2': 1.07, 'reg_fact_3': 2.22} || keep_prob = {'keep_prob_11': 0.8, 'keep_prob_12': 0.85, 'keep_prob_2': 0.9}
	Iteration  0 :	 J= 2.4412185907031048		Accuracy:  [array(0.231), array(0.2372), array(0.2278)]
	Iteration  50 :	 J= 0.462452044739073		Accuracy:  [array(0.91056), array(0.9197), array(0.9151)]
	Iteration  100 :	 J= 0.34568213083533605		Accuracy:  [array(0.93972), array(0.9425), array(0.9373)]
	Iteration  150 :	 J= 0.24618255473204143		Accuracy:  [array(0.94824), array(0.9498), array(0.9449)]
	Iteration  200 :	 J= 0.33180260080909824		Accuracy:  [array(0.95696), array(0.955), array(0.952)]
	Iteration  250 :	 J= 0.20778113251989241		Accuracy:  [array(0.96344), array(0.9597), array(0.9571)]
	Iteration  300 :	 J= 0.19706124606776398		Accuracy:  [array(0.96836), array(0.9644), array(0.9625)]
	Iteration  350 :	 J= 0.17629661857223317		Accuracy:  [array(0.97126), array(0.9656), array(0.9648)]
	Iteration  400 :	

**Some logs**

LOGS10000 - 10k iterations

LOGS1 and LOGS are for 1000 iterations

softmax function factors were changed in middle to check affect n accuracy. keep_prob and reg_param were changed in middle. learning_decay was also tried.


In [0]:
logs = [[reg_param,keep_prob],acc]
LOGS10000.append(logs)

In [0]:
LOGS10000

[[[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.07,
    'reg_fact_3': 2.22},
   {'keep_prob_11': 0.9, 'keep_prob_12': 1.07, 'keep_prob_2': 0.95}],
  [array(0.9857), array(0.9843)]]]

In [0]:
LOGS1

[[[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.31,
    'reg_fact_3': 2.22},
   {'keep_prob_11': 0.9, 'keep_prob_12': 0.86, 'keep_prob_2': 0.95}],
  [array(0.9786), array(0.9787)]]]

In [0]:
LOGS.append('adding rate decay (1/sqrt(epoch))')
LOGS

[[{'reg_fact_11': 1.19,
   'reg_fact_12': 1.05,
   'reg_fact_2': 1.2,
   'reg_fact_3': 1.2},
  [array(0.9761), array(0.9752)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.2,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.93, 'keep_prob_12': 0.87, 'keep_prob_2': 0.95}],
  [array(0.9823), array(0.9813)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.01,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.85, 'keep_prob_12': 0.87, 'keep_prob_2': 0.92}],
  [array(0.9815), array(0.9814)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.11,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.91, 'keep_prob_12': 0.87, 'keep_prob_2': 0.92}],
  [array(0.9826), array(0.9814)]],
 'changing softmax coeff from 5 to 1',
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.11,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.91, 'keep_prob_12': 0.87, 'keep_prob_2': 0.92}],
  [array(0.9835), array(0.9836)]],
 [[{'reg_fact_11':

In [0]:
LOGS

[[{'reg_fact_11': 1.19,
   'reg_fact_12': 1.05,
   'reg_fact_2': 1.2,
   'reg_fact_3': 1.2},
  [array(0.9761), array(0.9752)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.2,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.93, 'keep_prob_12': 0.87, 'keep_prob_2': 0.95}],
  [array(0.9823), array(0.9813)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.01,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.85, 'keep_prob_12': 0.87, 'keep_prob_2': 0.92}],
  [array(0.9815), array(0.9814)]],
 [[{'reg_fact_11': 1.19,
    'reg_fact_12': 1.05,
    'reg_fact_2': 1.11,
    'reg_fact_3': 1.2},
   {'keep_prob_11': 0.91, 'keep_prob_12': 0.87, 'keep_prob_2': 0.92}],
  [array(0.9826), array(0.9814)]]]

**Val and Test data prediction**

In [0]:
ans,_,__ = fwd_prop(X_val,Y_val,W11,b11,W12,b12,W2,b2,W3,b3)
ans = np.argmax(ans,axis=0)
np.mean((ans==y_val).astype(int))

0.9707

In [0]:
ans1,_,__ = fwd_prop(X_test,Y_test,W11,b11,W12,b12,W2,b2,W3,b3)
ans1 = np.argmax(ans1,axis=0)
np.mean((ans1==y_test).astype(int))

0.9699

Import time if required...

In [0]:
import time

In [0]:
LOG_REG_INC=[]